# Output Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine.

Authors: Johannes Giehl (jfg.eco@cbs.dk), Dana J. Hentschel (djh.eco@cbs.dk), Lucia Ciprian (luc.eco@cbs.dk)

## General settings

### Packages:

In [21]:
import pandas as pd
import os
import openpyxl
import numpy as np

In [22]:
def present_value_factor(n, r):
    """
    Calculate the present value factor of an annuity (Rentenbarwertfaktor).

    Parameters:
    n (int): The number of periods (time horizon).
    r (float): The discount rate (WACC).

    Returns:
    float: The present value factor of the annuity.
    """
    if r == 0:
        return n
    else:
        return (1 - (1 + r) ** -n) / r

### define parameters

In [23]:
#define if the model should use the last spine optimization results or a specific file
last_model_run = True # True or False

#set names of specific files
run_name = 'base_case'
specific_file_name = 'Output_exported' #+ run_name +'.xlsx'

#output files of this script
#file names to store the prepared output
output_prepared_export = 'output_'+ run_name +'_run.xlsx'

In [24]:
#parameters for present_value_factor calculation
time_horizon = 25  # Number of periods (years)
wacc = 0.1 * 1        # Discount rate
starting_year = "2020"
#os.chdir('C:/Users/luc.eco/OneDrive - CBS - Copenhagen Business School/Documents/GitHub/Nord_H2ub/Spine_Projects/03_output_data')

### define variables

In [25]:
#TBA

### File paths:

In [61]:
#get path of latest spine results
#parent folder
parent_folder_results = '../02_basic_energy_model/.spinetoolbox/items/exporter/output'
folders = [f for f in os.listdir(parent_folder_results) if os.path.isdir(os.path.join(parent_folder_results, f))]
if not folders:
    print("No folders found.")
else:
    latest_folder = max(folders, key=lambda x: os.path.getmtime(os.path.join(parent_folder_results, x)))
    latest_folder = "run@2024-10-14T13.04.57"
    latest_folder_path = os.path.join(parent_folder_results, latest_folder)
latest_folder_path = latest_folder_path.replace('\\', '/')
folder_path_results = latest_folder_path
folder_path_results += '/'

#get the information of the prepared input data that is used for the spine optimization
prepared_input_file_path = os.path.join('..', '01_input_data', '02_input_prepared')

#path to files from specific runs
path_specific_runs = '02_runs_EURO/01_output_raw/'

#prepared output data export to
output_file_path = '../03_output_data/01_basic_energy_model_outputs/'

In [62]:
#set the name of the relevant files
#input files for this script

#file name export from SpineToolbox
output_exported_file = 'Output_exported.xlsx'

#input file used for the optimization
data_from_inputs = '\methanol_Input_prepared.xlsx'

#this way of the output data preparation must be changed
#the information is manually added to the input xlsx and does not exist in the automated input generation
data_from_inputs_temporary = '\methanol_Input_prepared_for_output_temporary.xlsx'

In [63]:
#combine input path and files
#to input files
file_path_data_from_inputs  = prepared_input_file_path + data_from_inputs
full_path_data_from_inputs = os.path.abspath(os.path.join(os.getcwd(), file_path_data_from_inputs))

#to specific files
full_path_specific_files = path_specific_runs + specific_file_name

#for temporary appraoch
file_path_data_from_inputs_temporary  = prepared_input_file_path + data_from_inputs_temporary
full_path_data_from_inputs_temporary = os.path.abspath(os.path.join(os.getcwd(), file_path_data_from_inputs_temporary))

In [104]:
# Check if investment optimization was made (if any value in candidate_type is non-zero)
Investment_opt = False
input_prepared_file = pd.ExcelFile(file_path_data_from_inputs)
input_prepared_sheets = input_prepared_file.sheet_names

con_inv_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Connection_Inv_Parameters")
unit_inv_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Unit_Inv_Parameters")
stor_inv_params = pd.read_excel(full_path_data_from_inputs, sheet_name="Storage_Inv_Parameters")


if (con_inv_params['candidate_connections'] != 0).any() & \
   (con_inv_params['candidate_connections'].notna()).any() & \
   (con_inv_params['candidate_connections'] != '').any():
    investment_opt = True

# Check if there is any non-zero, non-empty, and non-NaN value in 'candidate_units'
if (unit_inv_params['candidate_units'] != 0).any() & \
   (unit_inv_params['candidate_units'].notna()).any() & \
   (unit_inv_params['candidate_units'] != '').any():
    investment_opt = True

# Check if there is any non-zero, non-empty, and non-NaN value in 'candidate_storages'
if (stor_inv_params['candidate_storages'] != 0).any() & \
   (stor_inv_params['candidate_storages'].notna()).any() & \
   (stor_inv_params['candidate_storages'] != '').any():
    investment_opt = True 

True


## Workflow of the data preparation

### Data Import

In [65]:
if last_model_run == True:
    df_output_raw = pd.read_excel(os.path.join(folder_path_results + output_exported_file), sheet_name=-1)
else:
    df_output_raw = pd.read_excel(full_path_specific_files)

df_PV_prices = pd.read_excel(file_path_data_from_inputs, sheet_name='Energy_prices')
df_model_definition = pd.read_excel(full_path_data_from_inputs, sheet_name='Definition')
df_units = pd.read_excel('../01_input_data/01_input_raw/methanol/Model_Data_Base_methanol.xlsx', sheet_name='Units')
df_storages = pd.read_excel('../01_input_data/01_input_raw/methanol/Model_Data_Base_methanol.xlsx', sheet_name='Storages')
#the next one seems to cause an issue
df_investment_costs = pd.read_excel('../01_input_data/01_input_raw/investment_cost_overview/investment_cost_overview.xlsx', sheet_name='Investment_Cost')
df_model_mapping = pd.read_excel('../01_input_data/02_input_prepared/methanol_object_mapping.xlsx', sheet_name='Object_Mapping')

C:\Users\luc.eco\AppData\Local\anaconda3\envs\py39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\luc.eco\AppData\Local\anaconda3\envs\py39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### data frame preparation

In [89]:
#create a copy of the original output DataFrame
df_output = df_output_raw.copy()

# Replace NaN values with empty strings in the first three rows
df_output.iloc[:3] = df_output.iloc[:3].fillna('')

# Combine the old header with the strings from the first three rows for each column
new_headers = df_output.columns + '_' + df_output.iloc[0] + '_' + df_output.iloc[1] + '_' + df_output.iloc[2]
old_headers = df_output.columns
# Set the new headers
df_output.columns = new_headers

# Drop the first three rows
#might be helpful bot not implemented now
#df_output = df_output.drop([0, 1, 2])

# Reset the index
df_output.reset_index(drop=True, inplace=True)
# Rename the first column to "timeseries"
df_output.columns.values[0] = "timeseries"

objective_connection_investment_costs_Model__
objective_connection_investment_costs_Model__


### data adjustments

In [90]:
#calculate revenues from PV sales on the wholesale market
selected_column_name = None
for column_index in range(len(df_output.columns)):
    if df_output.iloc[0, column_index] == 'power_line_Wholesale_Kasso' \
        and df_output.iloc[1, column_index] == 'to_node' \
        and df_output.iloc[2, column_index] == 'Power_Wholesale':
        selected_column_name = df_output.columns[column_index]
        break

# If connected to grid PV power is sold, else revenue is 0
if 'Power_Wholesale_Out' in df_PV_prices.columns:
    if selected_column_name:
        df_output['Revenue_from_PV'] = df_output[selected_column_name].iloc[3:] * df_PV_prices['Power_Wholesale_Out'].iloc[4]
    else:
        print("Column with specified headers not found in output.")

else: 
    df_output['Revenue_from_PV'] = 0 

objective_connection_investment_costs_Model__


In [98]:
#get total cost of the system
total_costs = df_output.filter(like='total_costs').iloc[3].values[0]
#get total revenue form PV power sale (times -1 is relevant as the input is structured that negative prices for exports reduce total cost). 
total_PV_revenue = df_output['Revenue_from_PV'].sum()*(-1)
#calculate cost without PV revenue
adjusted_costs = total_costs - (total_PV_revenue * (-1))

#create separate DataFrame for total and adjusted cost
df_system_cost_output = pd.DataFrame()
df_system_cost_output['Total_cost'] = total_costs
df_system_cost_output['PV_revenue'] = total_PV_revenue
df_system_cost_output['Total_adjusted_cost'] = adjusted_costs


5740617.55926496


In [99]:
# Identify columns to drop
columns_to_drop_1 = df_output.filter(like='total_costs').columns
# Drop the identified columns if any are found
if not columns_to_drop_1.empty:
    df_output.drop(columns=columns_to_drop_1, inplace=True)

#test this and implement an if check
columns_to_drop_2 = df_output.filter(like='unit_flow_op').columns
# Drop the identified columns if any are found
if not columns_to_drop_2.empty:
    df_output.drop(columns=columns_to_drop_2, inplace=True)

## calculate LCOE

calculation of levelized cost of energy

### calculate investment cost

In [100]:
df_model_definition

,Object_name,Category
0,Solar_Plant,unit
1,Electrolyzer,unit
2,CO2_Vaporizer,unit
3,Destilation_Tower,unit
4,Methanol_Reactor,unit
5,Steam_Plant,unit
6,Steam,node
7,Waste_Heat,node
8,Power_Kasso,node
9,Carbon_Dioxide,node


In [103]:
#implementation of either from the existing capacity as input from the model
#or if no input capacity is defined as max capacity used in the model

dict_investments = {}

if Investment_opt:
    for column_index in range(len(df_output.columns)):
        if df_output.iloc[0, column_index] == 'Model' \
        and "investment" in df_output.columns[column_index]:
            if "storage" in df_output.columns[column_index]:
                dict_investments["Storages"]  = df_output.iloc[3, column_index]
            elif "unit" in df_output.columns[column_index]:
                dict_investments["Units"]  = df_output.iloc[3, column_index]
            elif "connection" in df_output.columns[column_index]:
                dict_investments["Connections"]  = df_output.iloc[3, column_index]
    
else:
    for index_mapping, row_mapping in df_model_mapping.iterrows():
        investment = 0
        # Check if object is a unit
        for index_definition, row_definition in df_model_definition.iterrows():
            if row_definition["Category"] == "unit" and row_definition["Object_Name"]==row_mapping["Object_Name"]:
                for index_units, row_units in df_units.iterrows():   
                    # Find capacities of units
    
                    if row_mapping["Object_Name"] == "Destilation_Tower":
                        break # to not calculate the costs twice as investment costs are for Destilation Tower and Methanol Reactor combined
                    if row_mapping["Object_Name"] == row_units["Unit"]:
                        # If no capacity is given, find the maximum needed capacity calculated by the model
                        if pd.isnull(row_units["Cap_Input1_existing"]) and pd.isnull(row_units["Cap_Output1_existing"]):
                            for index_output, column_name in enumerate(df_output_raw.columns):
                                if "unit_flow" in column_name:
                                    # Exception: Capacity of Methanol plant, ToDo: Change capacity so it's not hard-coded 
                                    if row_mapping["Object_type"] == "Methanol_Plant":
                                        if df_output_raw.iloc[0, index_output] == "Destilation_Tower" and df_output_raw.iloc[2, index_output]=="Raw_Methanol":
                                            df_output_raw.loc[3:, column_name] = df_output_raw.loc[3:, column_name].astype(float)
                                            cap = df_output_raw.loc[3:,column_name].max()*0.795
                                            
                                    # Capacities for other units
                                    elif row_mapping["Object_Name"] == df_output_raw.iloc[0, index_output]:
                                        if "Power" in df_output_raw.iloc[2, index_output]:
                                            df_output_raw.loc[3:, column_name] = df_output_raw.loc[3:, column_name].astype(float)
                                            cap = df_output_raw.loc[3:,column_name].max()
        
                        # If capacity is known, choose it to calculate investment costs
                        else:   
                            if not pd.isnull(row_units["Cap_Input1_existing"]):
                                cap = row_units["Cap_Input1_existing"]
                            else:
                                cap = row_units["Cap_Output1_existing"]
    
                        # Calculate investment cost
                        for index_costs, row_costs in df_investment_costs.iterrows():
                            if row_costs["Object_type"] == row_mapping["Object_type"]:
                                costs_per_energy = row_costs[f"Investment_Cost [Euro/MW or MWh] Value {starting_year}"]
                        investment = cap * costs_per_energy
                        dict_investments[f"investment_{row_mapping['Object_Name']}"] = investment
                     
        
        # Check if object is a storage   
        if "storage" in row_mapping["Object_type"].lower():
            for index_storages, row_storages in df_storages.iterrows():
                # Find capacities of storages
                if row_storages["Storage"] == row_mapping["Object_Name"]:
                    # If no capacity is given, find the maximum needed capacity calculated by the model
                    if pd.isnull(row_storages["node_state_cap"]):  
                        for index_output, column_name in enumerate(df_output_raw.columns):
                            if "node_state" in column_name:
                                if row_mapping["Object_Name"] == df_output_raw.iloc[0, index_output]:
                                    df_output_raw.loc[3:, column_name] = df_output_raw.loc[3:, column_name].astype(float)
                                    cap = df_output_raw.loc[3:,column_name].max()
                                    
                    # If capacity is known, choose it to calculate investment costs
                    else:   
                        cap = row_storages["node_state_cap"]
    
                    # Calculate investment cost
                    for index_costs, row_costs in df_investment_costs.iterrows():
                        if row_costs["Object_type"] == row_mapping["Object_type"]:
                            costs_per_energy = row_costs[f"Investment_Cost [Euro/MW or MWh] Value {starting_year}"]
                    investment = cap * costs_per_energy
                    dict_investments[f"investment_{row_mapping['Object_Name']}"] = investment
        
        

    print(dict_investments)
total_investment = sum(dict_investments.values())
print(total_investment)

#electrolysis
#investment_electrolysis = 50000000

#methanol plant
#investment_methanol = 50000000

#hydrogen storage
#investment_hydrogen_storage = 50000000

#hydrogen storage
#investment_methanol_storage = 50000000

#further components like CO2 vaporizer, steam engine etc. 
#relevant to have a routine that identifies the units automatically. 


# changes made in excel sheets:
# methanol input prep file: object_type was added
# unit investment costs for detilation tower changed from see methanol reactor to values of methanol reactor
# To adjust:
# unit investment costs for each object type and uniform units (€/kW, €/MW ?) 
# delete Calculation of investment for destilation tower?
# Steam_Plant investments costs?

objective_storage_investment_costs_Model__
c 0
s 0
u 1202795.709203921
1202795.709203921


In [16]:
df_system_cost_output

,Total_cost,PV_revenue,Total_adjusted_cost
total_costs_toy__,379110.140382,1.415190e+07,14531010.50539


### slacks

In [17]:
# Calculate the slacks
node_slack_neg = df_output.filter(like='node_slack_neg')
node_slack_pos = df_output.filter(like='node_slack_pos')

node_slack_neg_numeric = node_slack_neg.apply(pd.to_numeric, errors='coerce')
node_slack_pos_numeric = node_slack_pos.apply(pd.to_numeric, errors='coerce')

node_slack_neg_sum = node_slack_neg_numeric.sum().sum()
node_slack_pos_sum = node_slack_pos_numeric.sum().sum()

total_slack = node_slack_neg_sum + node_slack_pos_sum

### variable costs

In [18]:
#get annual costs
annual_costs = df_system_cost_output.loc['total_costs_toy__', 'Total_adjusted_cost'] - total_slack

#annual cost including PV revenue
annual_costs_with_PV = df_system_cost_output.loc['total_costs_toy__', 'Total_cost'] - total_slack

### energy output

In [19]:
#energy output
energy_output = df_output.filter(like='Destilation_Tower_to_node_E-Methanol_Kasso')

# Convert strings to numbers, ignoring non-numeric values (relevant as first rows are strings)
energy_output_value = pd.to_numeric(energy_output.iloc[:,0], errors='coerce').sum() * 24

In [20]:
#calculation of the present value factor
pcf_value = present_value_factor(time_horizon, wacc)

LCOE = (total_investment + (annual_costs * pcf_value)) / (energy_output_value * pcf_value)
LCOE_PV = (total_investment + (annual_costs_with_PV * pcf_value)) / (energy_output_value * pcf_value)

In [21]:
#create a pandas data frame with all LCOE information

LCOE_GJ = LCOE/3.6
LCOE_t = LCOE_GJ*19.9

LCOE_GJ_PV = LCOE_PV/3.6
LCOE_PV_t = LCOE_GJ_PV*19.9

energy_output_value_t = (energy_output_value * 3.6) / 19.9


# Create a dictionary with the parameters
data_LCOE = {
    'LCOE [Euro/MWh]': LCOE,
    'LCOE [Euro/t]': LCOE_t,
    'total_investment': total_investment,
    'annual_costs': annual_costs,
    'energy_production [MWh]': energy_output_value,
    'energy_production [t]': energy_output_value_t,
    'pcf_value': pcf_value,
    'run_name': run_name
}

data_LCOE_PV = {
    'LCOE [Euro/MWh]': LCOE_PV,
    'LCOE [Euro/t]': LCOE_PV_t,
    'total_investment': total_investment,
    'annual_costs': annual_costs_with_PV,
    'energy_production [MWh]': energy_output_value,
    'energy_production [t]': energy_output_value_t,
    'pcf_value': pcf_value,
    'run_name': run_name + '_PV'
}

# Convert the dictionary to a DataFrame and set the index to the run name
df_LCOE_information = pd.DataFrame([data_LCOE])
df_LCOE_information.set_index('run_name', inplace=True)

df_LCOE_PV_information = pd.DataFrame([data_LCOE_PV])
df_LCOE_PV_information.set_index('run_name', inplace=True)

df_LCOE = pd.concat([df_LCOE_information, df_LCOE_PV_information])

### Creating one combined excel and export

In [22]:
'''with pd.ExcelWriter(output_file_path + output_prepared_export) as writer:
    df_output.to_excel(writer, sheet_name='flows_node_states')
    df_system_cost_output.to_excel(writer, sheet_name='system_costs')
    df_LCOE.to_excel(writer, sheet_name='LCOE')'''

"with pd.ExcelWriter(output_file_path + output_prepared_export) as writer:\n    df_output.to_excel(writer, sheet_name='flows_node_states')\n    df_system_cost_output.to_excel(writer, sheet_name='system_costs')\n    df_LCOE.to_excel(writer, sheet_name='LCOE')"

In [23]:
df_LCOE

,LCOE [Euro/MWh],LCOE [Euro/t],total_investment,annual_costs,energy_production [MWh],energy_production [t],pcf_value
run_name,,,,,,,
base_case,329.516384,1821.493343,3.898497e+08,1.452611e+07,174422.585305,31553.834528,9.07704
base_case_PV,248.380673,1372.993166,3.898497e+08,3.742133e+05,174422.585305,31553.834528,9.07704
